Summary , Tidy, Filtering and some Plots...on the way to start Analysis ...

In [ ]:
AADS<-read.csv("../input/AviationDataEnd2016UP.csv",colClasses = c("Event.Date"="Date"))

attach(AADS)
dim(AADS)

In [ ]:
# Libraries

library(plyr)
library(tidyr)
library(stringr)
library(stringi)
library(dplyr)
library(ggplot2)
library(gridExtra)
library(forcats)
library(Hmisc)

1- Summarizing the Dataset

In [ ]:
#dim(AADS)
#names(AADS)
#nrow(AADS)
#class(AADS)
#sapply(AADS[3,],class)
#length(unique(AADS$Accident.Number))
#nrow(AADS)
#table(AADS$Injury.Severity)
#table(AADS$Country,AADS$Total.Fatal.Injuries)

Before going ahead with summarizing the data, 
I did not like what they have done to Injury.Severity feature, maybe they imported from another source! 
It can be!  
 ...let us fix it.

In [ ]:
AADS$Injury.Severity<-gsub("Fatal","Fatalx",AADS$Injury.Severity)

AADS<-separate(AADS,Injury.Severity,c("Injury.Severity","Fatal"),sep="x")

AADS$Fatal<-gsub("\\(", "", AADS$Fatal)

AADS$Fatal<-gsub("\\)", "", AADS$Fatal)

Summarizing the Dataset

In [ ]:
#dim(AADS) 
#79293
#32
#names(AADS)
#nrow(AADS) 79293
#class(AADS) 'data.frame'
#sapply(AADS[3,],class)
#length(unique(AADS$Accident.Number))
#nrow(AADS) 79293
#table(AADS$Injury.Severity)
#Fatal    Incident   Non-Fatal Unavailable 
 #     15873        3175       60025         220 
#dim(AADS)
# 79293  32
#length(AADS$Country[AADS$Country == "United States"]) 74734

Let's focus on just United States, Because it is clear that the larger number of accidents recorded in NTSB database is for USA. 

**Tidy the data**

In [ ]:
# NA and Blank replacment..

AADS$Fatal[which(is.na(AADS$Fatal))]<-0

AADS$Fatal[AADS$Fatal==""]<-0

# Location variable needs to be separated...to get the State code of USA

AADS<-separate(AADS,Location,c("City","State"),sep=",")

# Here we are...

# Let's split the Event.Date variable As well to get more flexbilty..

AADS$Years<- format(as.POSIXct(strptime(AADS$Event.Date,"%Y-%m-%d",tz="")) ,format = "%Y")

AADS$Years[is.na(AADS$Years)] <- 1984

AADS$Months<- format(as.POSIXct(strptime(AADS$Event.Date,"%Y-%m-%d",tz="")) ,format = "%B")
# Same
AADS$Months[is.na(AADS$Months)] <- "April"

AADS$Days<- format(as.POSIXct(strptime(AADS$Event.Date,"%Y-%m-%d",tz="")) ,format = "%A")
# Same
AADS$Days[is.na(AADS$Days)] <- "Sunday"

AADS$Month.Number<- format(as.POSIXct(strptime(AADS$Event.Date,"%Y-%m-%d",tz="")) ,format = "%m")
# Same
AADS$Month.Number[is.na(AADS$Month.Number)] <- "04"

# Althought it was 100 % correct , sure need more investigation and verification.

# let's clean and Validate the Fatal before we go ahead... the question :
# is Fatal = Total.Fatal.Injuries??
AADS$Total.Fatal.Injuries[which(is.na(AADS$Total.Fatal.Injuries))]<-0
AADS$Total.Fatal.Injuries[AADS$Total.Fatal.Injuries==""]<-0
AADS$Fatal<-as.numeric(AADS$Fatal)
AADS$Total.Fatal.Injuries<-as.numeric(AADS$Total.Fatal.Injuries)

#sum(AADS$Total.Fatal.Injuries)
#sum(AADS$Fatal)
# 45543-45521
# 22... Oh.. wierd Yes?!
# it needs some investigation to find out the cause of this difference,,,
# Any way let's use Total.Fatal.Injuries ...because it is the dedicated variable of Fatal, Indeed.

#table(AADS$Total.Fatal.Injuries)
#length(unique(AADS$Accident.Number))


# we have some duplicated numbers of Fatal EX : 349 (2) , 154 (3)...Weird as well...
# lets figure out...checking the Accident Number...

# AADS$Accident.Number[30:40]
# WPR17LA034 #WPR17LA032 #ENG17IA005 #CEN17WA052B #CEN17WA052A #CEN17WA04
# CEN17LA047 #CEN17LA048 #CEN17FA046 #ANC17LA011 #CEN17FA045

# checking the Accident Number led me to conclusion that the majority of codes are 10 digits 
# but many of them ended by letters like A B and C Ex "ERA17FA027B and ERA17FA027A " 
# and the fatal number is the same ...
# so its looks like data updates..

# let's go then...

DTAB<-str_sub(AADS$Accident.Number,-1)
AADS$Dup.ACC<-DTAB
# let's delete rows contain A 
# let's delete rows contain B where C is exist

AADS <- AADS[!AADS$Dup.ACC == "A",]

# I will delete the rows contain  C ...3 of them are 0 fatal and one 3 fatal which is same in B version
# since i care only about the number of fatalties at this stage..
AADS <- AADS[!AADS$Dup.ACC == "C",]
# Let's clean the temp DATA

rm(DTAB)
AADS<-AADS[-38,-13]
#dim(AADS)
#names(AADS)

So..... United States, 

In [ ]:
NTSB.USA<- AADS %>% 
  filter(Country                 == "United States"
  ) %>% droplevels() %>% 
  select( Broad.Phase.of.Flight,Purpose.of.Flight, Total.Fatal.Injuries,
          Weather.Condition,Country,Amateur.Built,Injury.Severity,
          Aircraft.Category,Engine.Type,Aircraft.Category,Years,Months,
          Month.Number,Days,Longitude,Latitude)

In [ ]:
# dim(NTSB.USA) 
# 73656 15
#names(NTSB.USA)

Filtering

In [ ]:
NTSB.USA<- filter(NTSB.USA,Years %nin% c(1948,1962,1974,1977,1979,1981,2017))

NTSB.USA.Aircraft.Cat<- filter(NTSB.USA,Aircraft.Category %in% c( "Airplane","Balloon","Blimp","Glider",
                                                                  "Gyrocraft","Gyroplane","Helicopter",
                                                                  "Powered-Lift","Powered Parachute",
                                                                  "Rocket","Ultralight","Weight-Shift"))%>%
  droplevels()

NTSB.USA.Injury.Severity<- filter(NTSB.USA,Injury.Severity %in% c( "Fatal","Non-Fatal","Incident"))%>%
  droplevels()

NTSB.USA.Fatalities<-filter(NTSB.USA,Total.Fatal.Injuries %nin% c(0))%>%
  droplevels(as.factor(Total.Fatal.Injuries))

NTSB.USA.Fatalities.2up<-filter(NTSB.USA,Total.Fatal.Injuries %nin% c(0,1))%>%
  droplevels(as.factor(Total.Fatal.Injuries))

NTSB.USA.1.Fatality<-NTSB.USA%>% filter(
                                      Total.Fatal.Injuries    == 1        &
                                      Total.Fatal.Injuries   != ""        &
                                      Total.Fatal.Injuries   != "Unknown" &
                                      Aircraft.Category      != "Unknown" &
                                      Aircraft.Category      != ""        &
                                      Engine.Type            != "Unknown" &
                                      Engine.Type            != ""
) %>%
  droplevels()

NTSB.USA.1.Fatality.NoRec <- filter(NTSB.USA.1.Fatality,Engine.Type %nin% c("Reciprocating","Hybrid Rocket"))

NTSB.USA.Fatalities.For.Map <- subset(NTSB.USA.Fatalities, !is.na(Longitude))

Groups and Summaries

In [ ]:
NTSB.USA.by.Year <- summarise(group_by(NTSB.USA,Years),Num.Acc.per.Year=n())%>%
  arrange(desc(Num.Acc.per.Year))

NTSB.USA.by.Month<- summarise(group_by(NTSB.USA,Months),Num.Acc.by.Month=n()) %>%
  arrange(desc(Num.Acc.by.Month))

NTSB.USA.by.Day<- summarise(group_by(NTSB.USA,Days),Num.Acc.by.Day=n()) %>%
  arrange(desc(Num.Acc.by.Day))


NTSB.USA.by.Aircraft.Cat<- summarise(group_by(NTSB.USA.Aircraft.Cat,Aircraft.Category),Num.Acc.by.Aircraft.Cat=n()) %>%
  arrange(desc(Num.Acc.by.Aircraft.Cat))

NTSB.USA.by.Injury.Severity<- summarise(group_by(NTSB.USA.Injury.Severity,Injury.Severity),Num.Acc.by.Injury.Severity=n()) %>%
  arrange(desc(Num.Acc.by.Injury.Severity))

NTSB.USA.Fatal.Group<-summarise(group_by(NTSB.USA.Fatalities,Total.Fatal.Injuries),Fatalities.Group=n()) %>%
  arrange(desc(Fatalities.Group))
NTSB.USA.Fatal.Group$Total.Fatal.Injuries <- factor(NTSB.USA.Fatal.Group$Total.Fatal.Injuries)


NTSB.USA.Fatalities.2up.Group <- summarise(group_by(NTSB.USA.Fatalities.2up,Total.Fatal.Injuries),up2.Fatalities.Group=n()) %>%
  arrange(desc(up2.Fatalities.Group))

NTSB.USA.1.Fatality.Group <- summarise(group_by(NTSB.USA.1.Fatality,Aircraft.Category),One.Fatalitiy.Count=n())%>%
arrange(desc(One.Fatalitiy.Count))

# Plots #

Number of Accidents occurred per Year and Month

In [ ]:
######
Plot.Acc.Years<-ggplot(data = NTSB.USA.by.Year,
       aes(x=as.numeric(Years),
           y=Num.Acc.per.Year))+
  geom_line(size=2,col="darkred")+
  theme_light()+
 ggtitle("Number of Accidents occurred per Year")+
  labs(x="Year",
       y="Number of Accidents")+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))


######
NTSB.USA.by.Month$Months <- fct_inorder(NTSB.USA.by.Month$Months)
Plot.Acc.Month <- ggplot(data = NTSB.USA.by.Month,
       aes(x=Months,
           y=factor(Num.Acc.by.Month)))+
  geom_point(stat="Identity",fill="darkgreen",size=1.5)+
  theme_light()+
  ggtitle("Number of Accidents occurred by Month")+
  labs(x="Month",
       y="Number of Accidents")+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.3))

#######
Plot.Acc.Years.Points<-ggplot(data = NTSB.USA.by.Year,
                       aes(x=Years,
                           y=Num.Acc.per.Year))+
  geom_point(size=1,col="red")+
  theme_light()+
  ggtitle("Number of Accidents occurred per Year")+
  labs(x="Year",
       y="Number of Accidents")+
  theme(axis.text.x=element_text(size= 6, angle=90,hjust = 0.5))+
theme(axis.text.y=element_text(size= 5,hjust = 1))

#######
Plot.Acc.Month.Bars <- ggplot(data = NTSB.USA.by.Month,
                         aes(x=Months,
                             y=factor(Num.Acc.by.Month)))+
  geom_bar(stat="Identity",fill="darkgreen",size=5,width=0.5)+
  theme_light()+
  ggtitle("Number of Accidents occurred by Month")+
  labs(x="Month",
       y="Number of Accidents")+
  theme(axis.text.x=element_text(size= 6, angle=90,hjust = 0.5))
######
grid.arrange(Plot.Acc.Years,
             Plot.Acc.Month,
             Plot.Acc.Years.Points,
            Plot.Acc.Month.Bars,ncol=2)

Number of Accidents occurred by Day and Aircraft categoryies

In [ ]:
NTSB.USA.by.Day$Days <- fct_inorder(NTSB.USA.by.Day$Days)
Plot.Acc.Day <- ggplot(data = NTSB.USA.by.Day,
                         aes(x=Days,
                             y=factor(Num.Acc.by.Day)))+
  geom_point(stat="Identity",col="darkGreen",size=2.5)+
  theme_light()+
  ggtitle("Number of Accidents occurred by Day")+
  labs(x="Day",
       y="Number of Accidents")+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))+
  theme(axis.text.y=element_text(size= 5,hjust = 1))


NTSB.USA.by.Aircraft.Cat$Aircraft.Category <- fct_inorder(NTSB.USA.by.Aircraft.Cat$Aircraft.Category)
Plot.Acc.Aircraft.Cat <- ggplot(data = NTSB.USA.by.Aircraft.Cat,
                       aes(x=Aircraft.Category,
                           y=factor(Num.Acc.by.Aircraft.Cat)))+
  geom_point(stat="Identity",fill="darkblue",size=1.5)+
  theme_light()+
  ggtitle("umber of Accidents occurred by Aircraft Category")+
  labs(x="Aircraft Category",
       y="Number of Accidents")+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))+
  theme(axis.text.y=element_text(size= 5,hjust = 1))

grid.arrange(Plot.Acc.Day, Plot.Acc.Aircraft.Cat,ncol=2)

Let's check and plot the Injury.Severity ( Fatal, Non-Fatal, Incident )!

In [ ]:
NTSB.USA.by.Injury.Severity$Injury.Severity <- fct_inorder(NTSB.USA.by.Injury.Severity$Injury.Severity)
Plot.Acc.Injury.Severity <- ggplot(NTSB.USA.by.Injury.Severity,
                                   aes(x=Injury.Severity,
                                       y=factor(Num.Acc.by.Injury.Severity),fill=Injury.Severity))+
  geom_bar(stat="Identity",width = 0.2)+
  theme_light()+
  scale_fill_manual(values = c("Non-Fatal"="darkgreen",
                               "Fatal"= "darkred",
                               "Incident"="darkblue"))+
  ggtitle("Injury.Severity / Accidents")+
  labs(x="Injury Severity",
       y="Number of Accidents")

Plot.Acc.Injury.Severity

Let's plot the fatalities to see what to do in coming Kernels.

In [ ]:
Plot.Fatal.Group <- ggplot(NTSB.USA.Fatalities.2up.Group,
                           aes(x=factor(Total.Fatal.Injuries),
                               y=factor(up2.Fatalities.Group)))+
  geom_bar(stat="Identity",width = 0.2)+
  theme_light()+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))+
  ggtitle("Fatalities by Groups")+
  labs(x="Fatal Groups",
       y="Number of Accidents")

Plot.Fatal.Group

Let's find out the Aircraft Category and the Engine Type of Fatality = 1

In [ ]:
NTSB.USA.1.Fatality.Group$Aircraft.Category <- fct_inorder(NTSB.USA.1.Fatality.Group$Aircraft.Category)
plot.1.Fatality <- ggplot(NTSB.USA.1.Fatality.Group,
                              aes(x=Aircraft.Category,
                                  y=factor(One.Fatalitiy.Count)))+
  geom_bar(stat="identity",fill="darkgreen",width = 0.5)+
  theme_light()+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))+
  ggtitle("One Fatality distributions")+
  labs(x="Aircraft Category", y="Count of Accidents with 1 Fatality")
plot.1.Fatality

In [ ]:
Plot.Full.1.Fatality <-ggplot(NTSB.USA.1.Fatality,
                        aes(x=Aircraft.Category))+
  geom_bar(fill="darkgreen",width = 0.5)+ facet_wrap(~Engine.Type,ncol = 2)+
  theme_light()+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))+
  ggtitle("One Fatality distributions")+
  labs(x="Aircraft Category", y="Count of Accidents with 1 Fatality")
  
  
Plot.Full.1.Fatality

I wonder to know the number of accidents happened with the rest of airplane categories.

In [ ]:
Plot.Full.1.Fatality.NoRec <-ggplot(NTSB.USA.1.Fatality.NoRec,
                              aes(x=Aircraft.Category))+
  geom_bar(fill="darkgreen",width = 0.5)+ facet_wrap(~Engine.Type,ncol = 2)+
  theme_light()+
  theme(axis.text.x=element_text(size= 8, angle=90,hjust = 0.5))+
  ggtitle("One Fatality distributions")+
  labs(x="Aircraft Category", y="Count of Accidents with 1 Fatality")

Plot.Full.1.Fatality.NoRec

Airplanes using Reciprocating Engine Type have the majority (almost 100%) 
of the accidents number those have one fatality each.

However, you can check the link below to get an idea about Reciprocating Engine Type.
https://en.wikipedia.org/wiki/Reciprocating_engine

Here will be the End of this kernel for now, 
I would like to see the density of the Accidents on Map and check what is the correlation  between the Accidents and the weather conditions... 
So more will follow..
Please **upvote** if you liked it. :) 
Thanks